# Aggregating DataFrames in PySpark HW

First let's start up our PySpark instance

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [2]:
# Install pyspark
!pip install pyspark
# import findspark
# findspark.init()
import os
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 23.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=9776a58ac70ef00199d2485604601883ba4891e3a982a52e8de331a4b4cf45ec
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark
You are working with 1 core(s)


In [3]:
path = '/content/drive/MyDrive/Data Science Intake43/5. Spark/spark-scripts/section2/Datasets/'
os.listdir(path)

['people.json',
 'zomato.csv',
 'Weather.csv',
 'youtubevideos.csv',
 'googleplaystore.csv',
 'fifa19.csv',
 'students.csv',
 'users1.parquet',
 'users3.parquet',
 'users2.parquet',
 'nyc_air_bnb.csv',
 'rec-crime-pfa.csv',
 'supermarket_sales.csv',
 'Rep_vs_Dem_tweets.csv',
 'pga_tour_historical.csv',
 'uw-madison-courses']

## Read in the dataFrame for this Notebook

In [4]:
airbnb = spark.read.csv(path+'nyc_air_bnb.csv',inferSchema=True,header=True)

## About this dataset

This dataset describes the listing activity and metrics for Air BNB bookers in NYC, NY for 2019. Each line in the dataset is a booking. 

**Source:** https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data/data

Let's go ahead and view the first few records of the dataset so we know what we are working with.

In [5]:
airbnb.limit(5).toPandas()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,None,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


Now print the schema so we can make sure all the variables have the correct types

In [6]:
airbnb.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: string (nullable = true)
 |-- availability_365: integer (nullable = true)



Notice here that some of the columns that are obviously numeric have been incorrectly identified as "strings". Let's edit that. Otherwise we cannot aggregate any of the numeric columns.

In [7]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

df = airbnb.withColumn("price", airbnb["price"].cast(IntegerType())) \
        .withColumn("minimum_nights", airbnb["minimum_nights"].cast(IntegerType())) \
        .withColumn("number_of_reviews", airbnb["number_of_reviews"].cast(IntegerType())) \
        .withColumn("reviews_per_month", airbnb["reviews_per_month"].cast(IntegerType())) \
        .withColumn("calculated_host_listings_count", airbnb["calculated_host_listings_count"].cast(IntegerType()))
#QA
print(df.printSchema())
df.limit(5).toPandas()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: integer (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)

None


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.0,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.0,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.0,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.0,1,0


### Alright now we are ready to dig in!


### 1. How many rows are in this dataset?

In [8]:
# data shape
print((df.count(), len(df.columns)))
df.count()

(49079, 16)


49079

### 2. How many total reviews does each host have?

In [9]:
from pyspark.sql.functions import *

In [ ]:
airbnb.groupBy('host_id').agg({'number_of_reviews':'sum'}).show(5)

### 3. Show the min and max of all the numeric variables in the dataset

In [ ]:
num_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, IntegerType)]
num_cols

['price',
 'minimum_nights',
 'number_of_reviews',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

In [ ]:
airbnb.select(num_cols).summary('min', 'max').toPandas()

,summary,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,min,-73.99986,0,0,0,0,0
1,max,Private room,Private room,99,Private room,99,365


### 4. Which host had the highest number of reviews?

Only display the top result.

Bonus: format the column names

In [ ]:
hosts_rev = airbnb.groupBy('host_id').agg(sum(airbnb.number_of_reviews).alias('Rev_Sum'))
hosts_rev.orderBy(hosts_rev['Rev_Sum'].desc()).show(1)

+--------+-------+
| host_id|Rev_Sum|
+--------+-------+
|37312959| 2273.0|
+--------+-------+
only showing top 1 row



### 5. On average, how many nights did most hosts specify for a minimum?

In [ ]:
airbnb.groupBy('host_id').agg({'minimum_nights':'mean'}).show(10)

+-------+-------------------+
|host_id|avg(minimum_nights)|
+-------+-------------------+
| 716306|                1.0|
|1203500|                2.0|
| 368528|                1.0|
|1577493|                3.0|
|1390555|                1.0|
|1317588|               30.0|
|2472680|                4.0|
|2155832|                4.0|
|2426404|                6.0|
|2740824|               11.0|
+-------+-------------------+
only showing top 10 rows



### 6. What is the most expensive neighborhood to stay in on average?

Note: only show the one result

In [ ]:
neighb_price = airbnb.groupBy('neighbourhood_group').agg(mean(airbnb.price).alias('avg_price'))
neighb_price.orderBy('avg_price', ascending = False).show(1)

+-------------------+------------------+
|neighbourhood_group|         avg_price|
+-------------------+------------------+
|          Manhattan|196.74321570806705|
+-------------------+------------------+
only showing top 1 row



### 7. Display a two by two table that shows the average prices by room type (private and shared only) and neighborhood group (Manhattan and Brooklyn only)

In [ ]:
airbnb.filter(airbnb.room_type.isin('Shared room', 'Private room')).groupBy("room_type").pivot("neighbourhood_group", ["Manhattan", "Brooklyn"]).agg(mean(airbnb.price).alias("Avg Price")).show(10)

+------------+------------------+-----------------+
|   room_type|         Manhattan|         Brooklyn|
+------------+------------------+-----------------+
| Shared room| 89.06903765690376|50.52784503631961|
|Private room|116.05400302114803|76.47234042553191|
+------------+------------------+-----------------+



### Alright that's all folks!

### Great job!